In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from google.colab import files

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model2 = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [22]:
uploaded = files.upload()

image_path = next(iter(uploaded))
image = Image.open(image_path)

Saving superman.jpeg to superman.jpeg


In [23]:
inputs = processor(images=image, return_tensors="pt")
out = model2.generate(**inputs)

caption = processor.decode(out[0], skip_special_tokens=True)
caption

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'a man wearing a superman hoodie'

In [24]:
user_input = input("Enter additional text to describe your intent: ")

Enter additional text to describe your intent: show me t-shirts with same design


In [25]:
!pip install -U bitsandbytes

In [26]:
!pip install -q -U transformers accelerate

In [27]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16 )



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
final_query = f"Context: {caption}. User Query: {user_input}. Instruction: Generate a search query based mainly on context and type of website to search."
input_ids = tokenizer(final_query, return_tensors="pt").input_ids.to(device)

outputs = model.generate(
    input_ids,
    max_new_tokens=150,
    temperature=0.4,
    do_sample=True,
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Context: a man wearing a superman hoodie. User Query: show me t-shirts with same design. Instruction: Generate a search query based mainly on context and type of website to search. Contextual Search Query: "superman t-shirts" or "superman hoodie alternative" or "superman apparel" or "superman t-shirt designs" or "superman hoodie similar" or "superman clothing" or "superman t-shirt" or "superman hoodie replica" or "superman apparel similar" or "superman t-shirt similar" or "superman hoodie design" or "superman clothing similar" or "superman t-shirt design" or "superman hoodie alternative designs" or "superman apparel alternative" or "superman t-shirt alternative" or "superman hoodie design similar" or "superman clothing design" or "superman t-shirt design similar" or "superman hoodie similar design


In [29]:
import re
matches = re.findall(r'"(.*?)"', generated_text)

final_output = matches[0] if matches else "No valid query found."
print(final_output)


superman t-shirts


In [32]:
def perform_search(query, serpapi_key):
    params = {
        "engine": "google",
        "q": f'{query}',
        "api_key": serpapi_key,
        "num": 10,
        "hl": "en",
        "gl": "us"
    }
    search_url = "https://serpapi.com/search"
    response = requests.get(search_url, params=params)
    if response.status_code == 200:
        results = response.json()
        return results.get('organic_results', [])
    else:
        print("Error in API request:", response.status_code, response.text)
        return []
SERPAPI_KEY = "31dcf489be94603d1b32ab99f4c19e92f6333fb6e8c53ddf170d9b08548514ba"
search_results = perform_search(final_output, SERPAPI_KEY)
search_results

[{'position': 1,
  'title': 'Mens Superman Shirt',
  'link': 'https://www.amazon.com/Mens-Superman-Shirt/s?k=Mens+Superman+Shirt',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.amazon.com/Mens-Superman-Shirt/s%3Fk%3DMens%2BSuperman%2BShirt&ved=2ahUKEwiK6KKayY-JAxUcgIQIHWp-CKEQFnoECBQQAQ',
  'displayed_link': 'https://www.amazon.com › Mens-Superman-Shirt › k=M...',
  'thumbnail': 'https://serpapi.com/searches/670df29df9338826ff898df3/images/f0e4bf68796c964fd7f7ddc516bed37d8f3557106811fbd1c453c71475e2af42.jpeg',
  'favicon': 'https://serpapi.com/searches/670df29df9338826ff898df3/images/f0e4bf68796c964fd7f7ddc516bed37de0408deaf0e51dec9897558548b315e9.png',
  'snippet': "Superman Classic Logo T Shirt and Stickers, Unisex Adult T Shirt Collection · (2,373) · $17.99 ; Men's Superman Logo T-Shirt · (358) · $19.98 ; Superman Logo Men's ...",
  'snippet_highlighted_words': ['Superman',
   'T Shirt',
   'T Shirt',
   'Superman',
   'T',
   'Shi

so basically click on link under every tittle to see the the results of the search query
